In [1]:

import numpy as np
import optuna as optuna
import pandas as pd
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
import extract_correct_csv
from tqdm import tqdm
scaler = StandardScaler()


## logistic regression with sklearn

In [3]:
valid_subject = extract_correct_csv.extract_only_valid_subject()
if 49 in valid_subject:
    valid_subject.remove(49)
valid_subject

[1,
 2,
 4,
 5,
 6,
 7,
 13,
 15,
 16,
 17,
 21,
 22,
 23,
 26,
 27,
 28,
 31,
 32,
 33,
 41,
 43,
 44,
 45,
 46,
 47,
 48,
 50,
 51,
 52,
 54]

In [89]:
columns = ['subject', 'feature','fold', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)
index = 1
for x in valid_subject:



    types_ = ['hr', 'eda', 'pupil']
    for type_ in types_:
        X = pd.read_csv('data/features_4_2/'+type_+'/45.csv')
        X = X[48:]
        X = scaler.fit_transform(X)
        # creating train and test
        X = pd.DataFrame(X)
        X = X.reset_index().drop(columns=('index'))



        sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

        for i, (train_index, test_index) in enumerate(sss.split(X, y)):
            y = pd.DataFrame(y)
            y = y.reset_index().drop(columns=('index'))

            N_train = len(train_index)
            X_train = X.iloc[train_index, :]
            y_train =y.iloc[train_index, :]

            X_test = X.iloc[test_index, :]
            y_test =y.iloc[test_index, :]

            clf = LogisticRegression(random_state=123,max_iter=10000)
            clf.fit(X_train, np.squeeze(np.array(y_train)))

            train_ = clf.score(X_train, y_train)
            test_ = clf.score(X_test, y_test)
            dict_ = {'subject': x, 'feature': type_,'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
            results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
            index = index + 1


KeyboardInterrupt



## accuracy logistic classifier sklearn monophysio

In [ ]:
reg_mono = results
reg_mono

In [ ]:
columns = ['subject', 'fold']

In [ ]:
mean_regr=reg_mono.drop(columns=columns).groupby(by='feature', as_index=False).mean()
var_regr=reg_mono.drop(columns=columns).groupby(by='feature', as_index=False).var()

## mean accuracy over all subjs logistic classifier sklearn monophysio

In [ ]:
mean_regr

## var over all subjs logistic classifier sklearn monophysio


In [ ]:
var_regr

# Multi physio classifier

In [ ]:
subjects = extract_correct_csv.extract_only_valid_subject()
subjects

In [ ]:

columns_multi = ['subject','fold', 'train accuracy', 'test accuracy']
reg_multi = pd.DataFrame(columns = columns_multi)
index = 1

for sub in subjects:

    string_sub = extract_correct_csv.read_correct_subject_csv(sub)

    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]


    X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))
    X = pd.concat([X1, X2, X3], axis=1)
    X = X[48:]

    X = pd.DataFrame(X)
    X = X.reset_index().drop(columns=('index'))

    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        classifier = LogisticRegression(max_iter=10000,random_state=123)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': sub, 'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        reg_multi = pd.concat([reg_multi, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
        index = index + 1

## accuracy logistic classifier sklearn

In [ ]:
reg_multi

In [ ]:
columns = ['subject', 'fold']

## accuracy logistic classifier sklearn mean by subj

In [ ]:
grouped_df = reg_multi.groupby(by=['subject'], as_index=False).mean()
grouped_df

## mean accuracy over all subjs logistic classifier sklearn

In [ ]:
grouped_df.drop(columns=['subject']).mean()

## var over all subjs logistic classifier sklearn

In [ ]:
grouped_df.drop(columns=['subject']).var()

In [ ]:
print("test accuracy all physio\nmean: ",np.mean(np.array(mean_test)))
print('var: ',np.var(np.array(mean_test)))

## sklearn logistic regression OU roi features

In [4]:
columns_multi = ['subject','fold', 'train accuracy', 'test accuracy']
reg_multi = pd.DataFrame(columns = columns_multi)
index = 1

df_ = pd.read_csv('data/gaze/joined_fixation.csv')

In [7]:
df_['Fixation feature'][0]

'['

In [ ]:


for sub in subjects:

    string_sub = extract_correct_csv.read_correct_subject_csv(sub)


    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]


    X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))
    X = pd.concat([X1, X2, X3], axis=1)
    X = X[48:]

    X = pd.DataFrame(X)
    X = X.reset_index().drop(columns=('index'))

    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        classifier = LogisticRegression(max_iter=10000,random_state=123)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': sub, 'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        reg_multi = pd.concat([reg_multi, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)
        index = index + 1

# MLP

In [ ]:

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
valid_subject = extract_correct_csv.extract_only_valid_subject()
valid_subject.remove(50)
valid_subject.remove(51)

In [ ]:
# MLP mono signals
columns_mono = ['Subject', 'Feature', 'Train', 'Test']
MLP_df_mono = pd.DataFrame(columns=columns_mono)
types_ = ['hr', 'eda', 'pupil']
for sub in valid_subject:
    string_sub = extract_correct_csv.read_correct_subject_csv(sub)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]

    for type_ in types_:
        X = pd.read_csv('data/features_4_2/'+type_+'/'+str(sub)+'.csv')
        X = pd.DataFrame(scaler.fit_transform(X))
        X = X[48:]
        X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123, stratify=y)
        clf = MLPClassifier(hidden_layer_sizes=(30, 20), max_iter=3000, learning_rate='adaptive', random_state=123).fit(X_train, y_train)
        row_ = {'Subject': sub,
                'Feature': type_,
                'Train': clf.score(X_train, y_train),
                'Test':clf.score(X_test, y_test)}
        MLP_df_mono = pd.concat([MLP_df_mono, pd.DataFrame(data=row_, index=np.arange(1))], ignore_index=True)


In [ ]:
mean_mono = MLP_df_mono.groupby(by='Feature', as_index=False).mean()
std_mono = MLP_df_mono.groupby(by='Feature', as_index=False).std()

In [ ]:
from tqdm import tqdm

# test tuning MLP
first_layer_neurons = np.arange(10, 150, 10)
second_layer_neurons = np.arange(10, 150, 10)
third_layer_neurons = np.arange(10, 150, 10)


best_hyperP = pd.DataFrame(columns=['first layer', 'second layer', 'third layer', 'train', 'test'])

for first in first_layer_neurons:
    for second in second_layer_neurons:
        for third in third_layer_neurons:
            mean_test = []
            mean_train = []
            for sub in valid_subject:
                string_sub = extract_correct_csv.read_correct_subject_csv(sub)
                df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
                y = np.array(list([int (d > 2) for d in df_['rating']]))
                y = y[48:]
                X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
                X1 = pd.DataFrame(scaler.fit_transform(X1))
                X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
                X2 = pd.DataFrame(scaler.fit_transform(X2))
                X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
                X3 = pd.DataFrame(scaler.fit_transform(X3))
                X = pd.concat([X1, X2, X3], axis=1)
                X = X[48:]
                X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=123, stratify=y)
                clf = MLPClassifier(hidden_layer_sizes=(first, second, third),
                                    max_iter=3000,
                                    learning_rate='adaptive',
                                    random_state=123)\
                    .fit(X_train, y_train)
                mean_train.append(clf.score(X_train, y_train))
                mean_test.append(clf.score(X_test, y_test))

            row_dict = {'first layer': first,
                        'second layer': second,
                        'third layer': third,
                        'train': np.array(mean_train).mean(),
                        'test': np.array(mean_test).mean()}
            best_hyperP = pd.concat([best_hyperP, pd.DataFrame(data=row_dict, index=np.arange(1))], ignore_index=True)
best_hyperP.to_csv('output_mlp.csv')

In [ ]:
best_hyperP.loc[best_hyperP.test.idxmax()]

In [ ]:

columns = ['Subject', 'Train', 'Test']

mean_MLP_df = pd.DataFrame(columns=columns)
for sub in valid_subject:
    string_sub = extract_correct_csv.read_correct_subject_csv(sub)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    y = np.array(list([int (d > 2) for d in df_['rating']]))
    y = y[48:]


    X1 = pd.read_csv('data/features_4_2/hr/'+str(sub)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X2 = pd.read_csv('data/features_4_2/eda/'+str(sub)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(sub)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))

    X = pd.concat([X1, X2, X3], axis=1)
    X = X[48:]
    X_train, X_test,y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=123, stratify=y)
    clf = MLPClassifier(hidden_layer_sizes=(100, 50, 20), max_iter=1500, learning_rate='adaptive', random_state=123).fit(X_train, y_train)
    row_ = {'Subject': sub,
            'Train': clf.score(X_train, y_train),
            'Test':clf.score(X_test, y_test)}
    mean_MLP_df = pd.concat([mean_MLP_df, pd.DataFrame(data=row_, index=np.arange(1))], ignore_index=True)
mean_MLP_df

In [ ]:
mean_MLP_df.Test.mean()

## Logistic regression with morph level

In [2]:
# extract morph level functions
def extract_cs(df):
    cs1, cs2 = int(df[df.shock == True].picName.unique()[0][5]), int(df[df.shock == True].picName.unique()[1][5])
    return cs1, cs2


morph_pos = 11
morph_value = 15


def extract_threat_level(df):
    MORPH_POS = 11
    MORPH_VALUE = 15
    threat_person = []
    cs1_pic, cs2_pic = extract_cs(df)

    for i in df.iterrows():
        nome = i[1].picName
        try:
            if nome[5] == str(cs1_pic) or nome[5] == str(cs2_pic):
                threat_person.append(6)
                continue
            elif int(nome[5]):
                threat_person.append(1)
                continue
        except:
            if (int(nome[MORPH_POS]) == cs1_pic) or (int(nome[MORPH_POS]) == cs2_pic):
                if nome[MORPH_VALUE] == '2':
                    threat_person.append(5)

                elif nome[MORPH_VALUE] == '4':
                    threat_person.append(4)

                elif nome[MORPH_VALUE] == '6':
                    threat_person.append(3)

                elif nome[MORPH_VALUE] == '8':
                    threat_person.append(2)
                    continue
            else:
                if nome[MORPH_VALUE] == '2':
                    threat_person.append(2)

                elif nome[MORPH_VALUE] == '4':
                    threat_person.append(3)

                elif nome[MORPH_VALUE] == '6':
                    threat_person.append(4)

                elif nome[MORPH_VALUE] == '8':
                    threat_person.append(5)
    return threat_person

In [3]:
subjects = extract_correct_csv.extract_only_valid_subject()


In [20]:
columns = ['subject', 'feature', 'fold', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)

for x in subjects:
    string_sub = extract_correct_csv.read_correct_subject_csv(x)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    df_ = df_[48:]
    y = np.array(list([int (d > 2) for d in df_['rating']]))


    X1 = pd.read_csv('data/features_4_2/hr/'+str(x)+'.csv')
    X1 = pd.DataFrame(scaler.fit_transform(X1))
    X1 = X1[48:]
    print(X1.shape)
    X2 = pd.read_csv('data/features_4_2/eda/'+str(x)+'.csv')
    X2 = pd.DataFrame(scaler.fit_transform(X2))
    X2 = X2[48:]
    print(X2.shape)
    X3 = pd.read_csv('data/features_4_2/pupil/'+str(x)+'.csv')
    X3 = pd.DataFrame(scaler.fit_transform(X3))
    X3 = X3[48:]
    print(X3.shape)
    img = np.array(extract_threat_level(df_))
    morph_level = np.zeros((img.size, img.max()))
    morph_level[np.arange(img.size), img-1] = 1
    X4 = morph_level.astype(int)
    X4 = pd.DataFrame(X4)
    print(X4)
    X = pd.concat([X1, X2, X3, X4], axis=1)

    #print(X.shape)
    break
    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        clf = LogisticRegression(random_state=123,max_iter=10000)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': x, 'feature': 'morph level' ,'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)

(112, 24)
(112, 24)
(112, 28)
(112, 6)
(160, 82)


In [5]:
results.groupby(by='subject').mean(numeric_only=True)

,train accuracy,test accuracy
subject,,
2,0.794007,0.811594
4,0.670412,0.666667
10,0.696629,0.782609
41,0.771536,0.623188
43,0.868914,0.855072
44,0.707865,0.579710
45,0.921348,0.913043
46,0.655431,0.637681
47,0.910112,0.840580


## Mean over all subjects

In [6]:
results.drop(columns=['subject', 'fold']).mean(numeric_only=True)

train accuracy    0.797312
test accuracy     0.774936
dtype: float64

In [7]:
results.drop(columns=['subject', 'fold']).var(numeric_only=True)

train accuracy    0.008994
test accuracy     0.019713
dtype: float64

## Logistic regression with all signals

In [ ]:
columns = ['subject', 'feature', 'fold', 'train accuracy', 'test accuracy']
results = pd.DataFrame(columns = columns)

for x in subjects:
    string_sub = extract_correct_csv.read_correct_subject_csv(x)
    df_ = pd.read_csv('data/LookAtMe_0'+string_sub+'.csv', sep='\t')
    df_ = df_[48:]
    y = np.array(list([int (d > 2) for d in df_['rating']]))


    img = np.array(extract_threat_level(df_))
    morph_level = np.zeros((img.size, img.max()))
    morph_level[np.arange(img.size), img-1] = 1
    X = morph_level.astype(int)
    X = pd.DataFrame(X)
    X = X.reset_index().drop(columns='index')

    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=123)

    for i, (train_index, test_index) in enumerate(sss.split(X, y)):
        y = pd.DataFrame(y)
        y = y.reset_index().drop(columns=('index'))

        N_train = len(train_index)
        X_train = X.iloc[train_index, :]
        y_train =y.iloc[train_index, :]

        X_test = X.iloc[test_index, :]
        y_test =y.iloc[test_index, :]

        clf = LogisticRegression(random_state=123,max_iter=10000)
        clf.fit(X_train, np.squeeze(np.array(y_train)))

        train_ = clf.score(X_train, y_train)
        test_ = clf.score(X_test, y_test)
        dict_ = {'subject': x, 'feature': 'morph level' ,'fold':int(i), 'train accuracy': train_, 'test accuracy': test_}
        results = pd.concat([results, pd.DataFrame(data=dict_, index=np.arange(1))], ignore_index=True)

## Results with our model

In [8]:
mono_im = pd.read_csv('output/FA/FA_only_image.csv')
mono_im

,subject,k,fold,train,test
0,2,2,0,0.505618,0.478261
1,2,2,1,0.617978,0.739130
2,2,2,2,0.494382,0.521739
3,2,3,0,0.213483,0.391304
4,2,3,1,0.786517,0.869565
...,...,...,...,...,...
235,55,5,1,0.674157,0.652174
236,55,5,2,0.573034,0.521739
237,55,6,0,0.292135,0.304348
238,55,6,1,0.550562,0.521739


In [ ]:
complete_image = pd.read_csv('output/FA/FA_kcv_norm_image_new.csv')